In [ ]:
from pyincore import IncoreClient, FragilityService, RestorationService, MappingSet
from pyincore.analyses.pipelinedamagerepairrate import PipelineDamageRepairRate, PipelineUtil

In [ ]:
client = IncoreClient()

### Step 1: Calculate pipeline damage with repair rate for Shelby county, TN 

In [ ]:
# New madrid earthquake using Atkinson Boore 1995
hazard_type = "earthquake"
hazard_id = "5b902cb273c3371e1236b36b"

# Water Buried Pipeline inventory in Shelby county, TN
pipeline_dataset_id = "5a284f28c7d30d13bc081d14"

# pipeline fragility mapping
mapping_id = "5b47c227337d4a38464efea8"

# Geology dataset
liq_geology_dataset_id = "5a284f53c7d30d13bc08249c"
liq_fragility_key = "pgd"
use_liq = True

# Specify the result name
result_name = "pipeline_result"

In [ ]:
# Create pipeline damage with repair rate
pipeline_dmg_w_rr = PipelineDamageRepairRate(client)

# Load pipeline inventory as input datasets
pipeline_dmg_w_rr.load_remote_input_dataset("pipeline", pipeline_dataset_id)

# Load fragility mapping
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
pipeline_dmg_w_rr.set_input_dataset("dfr3_mapping_set", mapping_set)

In [ ]:
# Set analysis parameters
pipeline_dmg_w_rr.set_parameter("result_name", result_name)
pipeline_dmg_w_rr.set_parameter("hazard_type", hazard_type)
pipeline_dmg_w_rr.set_parameter("hazard_id", hazard_id)
pipeline_dmg_w_rr.set_parameter("liquefaction_fragility_key", liq_fragility_key)
pipeline_dmg_w_rr.set_parameter("liquefaction_geology_dataset_id",liq_geology_dataset_id)
pipeline_dmg_w_rr.set_parameter("use_liquefaction", use_liq)
pipeline_dmg_w_rr.set_parameter("num_cpu", 4)

# Run pipeline  damage analysis
pipeline_dmg_w_rr.run_analysis()
# Retrieve result dataset
pipeline_dmg_result = pipeline_dmg_w_rr.get_output_dataset("result")

### Step 2: Run pipeline restoration analysis by chaining output from Step 1

In [ ]:
from pyincore.analyses.pipelinerestoration import PipelineRestoration

pipeline_restoration = PipelineRestoration(client)

In [ ]:
pipeline_restoration.load_remote_input_dataset("pipeline", "5a284f28c7d30d13bc081d14")
pipeline_restoration.set_input_dataset("pipeline_damage", pipeline_dmg_result)

In [ ]:
 # Load fragility mapping
restoration_service = RestorationService(client)
mapping_set = MappingSet(restoration_service.get_mapping("61f35f09903e515036cee106"))
pipeline_restoration.set_input_dataset('dfr3_mapping_set', mapping_set)

pipeline_restoration.set_parameter("result_name", "pipeline_restoration_times")

pipeline_restoration.set_parameter("restoration_key", "Restoration ID Code")
pipeline_restoration.set_parameter("num_available_workers", 4)
pipeline_restoration.set_parameter("num_cpu", 4)

# Run pipeline restoration analysis
pipeline_restoration.run_analysis()

In [ ]:
# Get the result dataset
pipeline_restoration_result = pipeline_restoration.get_output_dataset("pipeline_restoration")
# Convert dataset to Pandas DataFrame
df = pipeline_restoration_result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()